#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.annotator.pos_tagger import POSTagger

#### **2. 加载模型**

加载大模型词性标注 API 接口和 HanLP

In [2]:
# === 加载模型：LLM PKU 赋码集 ===

# 语种：汉语
lang = 'chinese'

# 赋码集：PKU
# https://hanlp.hankcs.com/docs/annotations/pos/pku.html
tagset = 'pku'

# 标注模式：LLM API
mode = 'llm'

# 大模型：kimi-k2.5 | glm-5 | deepseek-v3.2 | qwen3-max
llm_models = ['kimi-k2.5', 'glm-5']

# 登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********

llm_tagger = {}
for model in llm_models:
    llm_tagger[model] = POSTagger(
        lang=lang,
        tagset=tagset,
        mode=mode,
        llm_model=model,
    )
print('LLM 中文词性标注模型加载完毕！')

LLM 中文词性标注模型加载完毕！


In [3]:
# === 加载模型：HanLP PKU 赋码集 ===

# 标注模式：HanlP 本地模型
mode = 'local'

# HanLP PKU 模型：PKU_POS_ELECTRA_SMALL
# https://hanlp.hankcs.com/docs/api/hanlp/pretrained/pos.html
# 首次加载需从网络将模型下载至本地

hanlp_tagger = POSTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
)
print('HanLP PKU 中文词性标注模型加载完毕！')

HanLP PKU 中文词性标注模型加载完毕！


#### **3. 词性标注**

使用大模型和 HanLP 标注汉语词性

In [4]:
# === 测试数据 ===

# 选自《鹿鼎记》

zh_text = """
韦小宝骂道：“辣块妈妈，臭小娘皮，你还倔强！ 睁开眼睛来，瞧着我！
他们欠了你钱，不敢不依，如果推三阻四，你就说我会去跟上书房总管乌老公算帐。
陈近南脸一板，道：“你给我正正经经的，少油嘴滑舌。
你老实说了出来，我立刻给你解药。
"""

In [5]:
# === 调用大模型 API 和 HanLP：开始词性标注 ===

# --- 注意 ---
# 为节省 API 调用成本
# 大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果

# 若需测试 API 连接是否正常
# 可更换测试数据，重新标注

print(f'=== 中文词性标注 ===\n')

# 逐行遍历所有数据
for index, text in enumerate(zh_text.strip().splitlines()):
    print(f'[ID]: {index+1:05d}')
    print(text)
    print(f'{"-" * 60}')

    # 大模型词性标注
    for model in llm_models:
        print(f'[{model}]:')
        llm_anno = llm_tagger[model].tag(text.strip())
        llm_tags = [(tok, pos) for (tok, pos) in zip(llm_anno['tok'], llm_anno['pos'])]
        print(llm_tags)
        print(f'{'-' * 60}')

    # HanLP 词性标注
    print(f'[hanlp]:')
    hanlp_anno = hanlp_tagger.tag(text.strip())
    hanlp_tags = [(tok, pos) for (tok, pos) in zip(hanlp_anno['tok'], hanlp_anno['pos'])]
    print(hanlp_tags)
    print(f'{'-' * 60}\n')

=== 中文词性标注 ===

[ID]: 00001
韦小宝骂道：“辣块妈妈，臭小娘皮，你还倔强！ 睁开眼睛来，瞧着我！
------------------------------------------------------------
[kimi-k2.5]:
[('韦小宝', 'nr'), ('骂道', 'v'), ('：', 'w'), ('“', 'w'), ('辣块妈妈', 'l'), ('，', 'w'), ('臭', 'a'), ('小娘皮', 'n'), ('，', 'w'), ('你', 'r'), ('还', 'd'), ('倔强', 'a'), ('！', 'w'), ('睁开', 'v'), ('眼睛', 'n'), ('来', 'v'), ('，', 'w'), ('瞧着', 'v'), ('我', 'r'), ('！', 'w'), ('”', 'w')]
------------------------------------------------------------
[glm-5]:
[('韦小宝', 'nr'), ('骂', 'v'), ('道', 'v'), ('：', 'w'), ('“', 'w'), ('辣块妈妈', 'l'), ('，', 'w'), ('臭', 'a'), ('小娘皮', 'n'), ('，', 'w'), ('你', 'r'), ('还', 'd'), ('倔强', 'a'), ('！', 'w'), ('睁', 'v'), ('开', 'v'), ('眼睛', 'n'), ('来', 'v'), ('，', 'w'), ('瞧', 'v'), ('着', 'u'), ('我', 'r'), ('！', 'w'), ('”', 'w')]
------------------------------------------------------------
[hanlp]:
[('韦小宝', 'nr'), ('骂道', 'v'), ('：', 'w'), ('“', 'w'), ('辣', 'a'), ('块', 'q'), ('妈妈', 'n'), ('，', 'w'), ('臭', 'a'), ('小娘皮', 'n'), ('，', 'w'), ('你', 'r'), ('还', '